In [2]:
# !wget https://f000.backblazeb2.com/file/malay-dataset/dumping/singlish/singlish.txt
# !wget https://f000.backblazeb2.com/file/malay-dataset/dumping/singlish/sg-news.txt
# !wget https://f000.backblazeb2.com/file/malay-dataset/dumping/twitter/manglish.json

files = ['singlish.txt',
        'sg-news.txt',]

In [3]:
import os
import string

vocabs = list(string.ascii_lowercase + string.digits) + [' ']
directory = './'

In [7]:
import unicodedata
import re
import itertools

def preprocessing_text(string):
    
    string = re.sub(
        'http\\S+|www.\\S+',
        '',
        ' '.join(
            [
                word
                for word in string.split()
                if word.find('#') < 0 and word.find('@') < 0
            ]
        ),
    )
        
    string = unicodedata.normalize('NFC', string.lower())
    string = ''.join([c if c in vocabs else ' ' for c in string])
    string = re.sub(r'[ ]+', ' ', string).strip()
    string = (
        ''.join(''.join(s)[:2] for _, s in itertools.groupby(string))
    )
    return string

In [8]:
texts = []

for f in files:
    print(f)
    with open(os.path.join(directory, f)) as fopen:
        text = list(filter(None, fopen.read().split('\n')))
        texts.extend(text)

singlish.txt
sg-news.txt


In [9]:
import json

with open('bahasa-asr-train-combined.json') as fopen:
    data = json.load(fopen)
    
texts.extend(data['Y'])

with open('bahasa-asr-test.json') as fopen:
    data = json.load(fopen)
    
texts.extend(data['Y'])

In [10]:
with open('manglish.json') as fopen:
    t = json.load(fopen)

In [11]:
len(t)

908120

In [18]:
import mp
from tqdm import tqdm

def loop(texts):
    texts, _ = texts
    cleaned_texts = []
    for i in tqdm(range(len(texts))):
        t = preprocessing_text(texts[i])
        if len(t):
            cleaned_texts.append(t)
    return cleaned_texts

cleaned_texts = mp.multiprocessing(texts, loop, cores = 16)

100%|██████████| 1460357/1460357 [03:56<00:00, 6176.70it/s] 


In [20]:
len(texts), len(cleaned_texts)

(23365724, 23135501)

In [21]:
with open('text.txt', 'w') as fopen:
    fopen.write('\n'.join(cleaned_texts))

In [22]:
!./kenlm/build/bin/lmplz --text text.txt --arpa out.arpa -o 3 --prune 0 1 1

=== 1/5 Counting and sorting n-grams ===
Reading /home/husein/malaya-speech/text.txt
----5---10---15---20---25---30---35---40---45---50---55---60---65---70---75---80---85---90---95--100
****************************************************************************************************
Unigram tokens 273001650 types 695765
=== 2/5 Calculating and sorting adjusted counts ===
Chain sizes: 1:8349180 2:75179769856 3:140962070528
Statistics:
1 695765 D1=0.684986 D2=1.02996 D3+=1.33605
2 8959693/21423984 D1=0.720398 D2=1.0641 D3+=1.36516
3 24790903/89937448 D1=0.721381 D2=1.3111 D3+=1.47455
Memory estimate for binary LM:
type     MB
probing 647 assuming -p 1.5
probing 701 assuming -r models -p 1.5
trie    281 without quantization
trie    163 assuming -q 8 -b 8 quantization 
trie    264 assuming -a 22 array pointer compression
trie    146 assuming -a 22 -q 8 -b 8 array pointer compression and quantization
=== 3/5 Calculating and sorting initial probabilities ===
Chain sizes: 1:8349180 2:14335

In [23]:
!./kenlm/build/bin/build_binary -q 8 -b 7 -a 256 trie out.arpa out.trie.klm

Reading out.arpa
----5---10---15---20---25---30---35---40---45---50---55---60---65---70---75---80---85---90---95--100
****************************************************************************************************
Identifying n-grams omitted by SRI
----5---10---15---20---25---30---35---40---45---50---55---60---65---70---75---80---85---90---95--100
****************************************************************************************************
Quantizing
----5---10---15---20---25---30---35---40---45---50---55---60---65---70---75---80---85---90---95--100
****************************************************************************************************
Writing trie
----5---10---15---20---25---30---35---40---45---50---55---60---65---70---75---80---85---90---95--100
****************************************************************************************************
SUCCESS


In [24]:
!rm text.txt out.arpa

In [26]:
b2_application_key_id = os.environ['b2_application_key_id']
b2_application_key = os.environ['b2_application_key']

In [27]:
from b2sdk.v1 import *
info = InMemoryAccountInfo()
b2_api = B2Api(info)
application_key_id = b2_application_key_id
application_key = b2_application_key
b2_api.authorize_account("production", application_key_id, application_key)
file_info = {'how': 'good-file'}
b2_bucket = b2_api.get_bucket_by_name('malaya-speech-model')

In [28]:
outPutname = 'language-model/manglish/model.trie.klm'
b2_bucket.upload_local_file(
    local_file='out.trie.klm',
    file_name=outPutname,
    file_infos=file_info,
)